#### Assignment 11
Using section 8.1 in Deep Learning with Python as a guide, implement an LSTM text generator. Train the model on the Enron corpus or a text source of your choice. Save the model and generate 20 examples to the results directory of dsc650/assignments/assignment11/.

In [26]:
import keras
import numpy as np
from pathlib import Path
import os
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
fullPath = os.path.abspath("/home/jovyan/dsc650/dsc650/assignments/assignment11/" + 'LOTR_text.txt')
fullPath
#https://stackoverflow.com/questions/60166043/can-tf-keras-utils-get-file-be-used-to-load-local-zip-files

'/home/jovyan/dsc650/dsc650/assignments/assignment11/LOTR_text.txt'

In [16]:
path_LOTR = keras.utils.get_file(
    'LOTR_text.txt','file://'+fullPath)

2596864/2592418 [==============================] - 0s 0us/step


In [17]:
text_LOTR = open(path_LOTR).read().lower()
print('Corpus length:', len(text_LOTR))

Corpus length: 2579785


In [19]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text_LOTR) - maxlen, step):
    sentences.append(text_LOTR[i: i + maxlen])
    next_chars.append(text_LOTR[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text_LOTR)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 859909
Unique characters: 68
Vectorization...


In [20]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [21]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [22]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
fileName = results_dir.joinpath('LSTM_LOTR.h5')
model.save(fileName)

In [33]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text_LOTR) - maxlen - 1)
    generated_text = text_LOTR[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
    # from teams convo
    example_filename = 'LSTM_Example-Epoch_' + str(epoch) + '_Temp_' + str(temperature) + '.txt'
    with open(results_dir.joinpath(example_filename), 'w') as file:
        file.writelines(generated_text)
        
        print()

epoch 1
6719/6719 [==============================] - 854s 127ms/step - loss: 1.3393
--- Generating with seed: "ce do you come? and whither do you go? what is your business"
------ temperature: 0.2
ce do you come? and whither do you go? what is your business and the stream, and there was a shadow of the day of the lord of the shire of the lord of the short than the mountains of the stone that was the house of the company was beast by the shire of the stone of the shadow of the shadow of the shire of the stone of the road and the stones. the stream had the stone of the stranger of the darkness and strength the strength and fell and stretched and strea------ temperature: 0.5
s and strength the strength and fell and stretched and stream was was the store and stone could do with a long break and walking on the ford that it went under but rode to deats. so i got a great houses of nights. and the long was that hope to perhaps and grows and very a men was not all singing, and so not sam from t

<ipython-input-22-66c6eecb696d>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


      comies as on the o! he ipsetlels he lempted_ for!'
     'a springs ip, if you, if ! i take it in elberminarr is scurst about sleep.'
     `what moundingitarms, or pay, and never? y
epoch 4
6719/6719 [==============================] - 859s 128ms/step - loss: 1.3043
--- Generating with seed: "is entmoot?' pippin ventured to ask.
     'hoo, eh? entmoot?"
------ temperature: 0.2
is entmoot?' pippin ventured to ask.
     'hoo, eh? entmoot?' said gandalf. `i should have the thing they were so that i do not come and the stones of the for the stones of the world of the stones of the lower of the stones of the stones of the grey stream and stream and stream and looked and streng
     'we should be seen the stream of the woods of the grey of the land of the stone of the land of the stones of the stones of the world of the way they stood------ temperature: 0.5
 the stones of the stones of the world of the way they stood behind his shadows was and desperate of upon the broken and stream and 

ValueError: pvals < 0, pvals > 1 or pvals contains NaNs